In [1]:
import cv2
import time
import pyttsx3
from num2words import num2words


cap = cv2.VideoCapture(0)


if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()


face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


vertical_area = [(320, 0), (320, 480)]


counter = 0


person_crossed = False


engine = pyttsx3.init()


def speak_number(number):
    word = num2words(number, to='ordinal')  
    engine.say(word)
    engine.runAndWait()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame.")
        break

    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    
    for (x, y, w, h) in faces:
        if x < vertical_area[0][0] < x + w:
            if not person_crossed:
                counter += 1
                speak_number(counter)
                person_crossed = True
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        else:
            person_crossed = False

    
    count_word = num2words(counter, to='ordinal')  

    
    cv2.putText(frame, f'Persons Crossing Vertical Area: {count_word}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    
    cv2.line(frame, vertical_area[0], vertical_area[1], (0, 0, 255), 2)

    
    cv2.imshow('Frame', frame)

    
    if person_crossed:
        time.sleep(0.25)

    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
